Enunciado: clase Cuenta con herencia
Desarrollar un programa que modele una cuenta bancaria que tiene los siguientes atributos, que deben ser de acceso protegido:
- Saldo, de tipo float.
- Número de consignaciones con valor inicial cero, de tipo int.
- Número de retiros con valor inicial cero, de tipo int.
- Tasa anual (porcentaje), de tipo float.
- Comisión mensual con valor inicial cero, de tipo float.

La clase Cuenta tiene un constructor que inicializa los atributos saldo y tasa anual con valores pasados como parámetros. La clase Cuenta tiene los siguientes métodos:
- Consignar una cantidad de dinero en la cuenta actualizando su saldo.
- Retirar una cantidad de dinero en la cuenta actualizando su saldo. El valor a retirar no debe superar el saldo.
- Calcular el interés mensual de la cuenta y actualiza el saldo correspondiente.
- Extracto mensual: actualiza el saldo restándole la comisión mensual y calculando el interés mensual correspondiente (invoca el método anterior).
- Imprimir: muestra en pantalla los valores de los atributos.

La clase Cuenta tiene dos clases hijas:
- Cuenta de ahorros: posee un atributo para determinar si la cuenta de ahorros está activa (tipo boolean). Si el saldo es menor a $10000, la cuenta está inactiva, en caso contrario se considera activa. Los siguientes métodos se redefinen:
- Consignar: se puede consignar dinero si la cuenta está activa. Debe invocar al método heredado.
- Retirar: es posible retirar dinero si la cuenta está activa. Debe invocar al método heredado.
- Extracto mensual: si el número de retiros es mayor que 4, por cada retiro adicional, se cobra $1000 como comisión mensual. Al generar el extracto, se determina si la cuenta está activa o no con el saldo.

In [1]:
class Cuenta:
    def __init__(self, saldo, tasa_anual):
        self._saldo = saldo
        self._tasa_anual = tasa_anual
        self._num_consignaciones = 0
        self._num_retiros = 0
        self._comision_mensual = 0.0
        self._activa = True

    def consignar(self, cantidad):
        if cantidad > 0:
            self._saldo += cantidad
            self._num_consignaciones += 1
            print("Consignación realizada exitosamente.")
        else:
            print("Cantidad inválida para consignación.")

    def retirar(self, cantidad):
        if cantidad > 0 and cantidad <= self._saldo:
            self._saldo -= cantidad
            self._num_retiros += 1
            print("Retiro realizado exitosamente.")
        else:
            print("Saldo insuficiente o cantidad inválida.")

    def calcular_interes_mensual(self):
        interes_mensual = self._saldo * (self._tasa_anual / 12 / 100)
        self._saldo += interes_mensual

    def extracto_mensual(self):
        self._saldo -= self._comision_mensual
        self.calcular_interes_mensual()

    def imprimir(self):
        print(f"Saldo: {self._saldo:.2f}")
        print(f"Consignaciones: {self._num_consignaciones}")
        print(f"Retiros: {self._num_retiros}")
        print(f"Tasa anual: {self._tasa_anual}%")
        print(f"Comisión mensual: {self._comision_mensual:.2f}")
        print(f"Cuenta activa: {self._activa}")


In [2]:
class CuentaAhorros(Cuenta):
    def __init__(self, saldo, tasa_anual):
        super().__init__(saldo, tasa_anual)
        self._activa = saldo >= 10000

    def consignar(self, cantidad):
        if self._activa:
            super().consignar(cantidad)
            self._activa = self._saldo >= 10000
        else:
            print("Cuenta inactiva. No se puede consignar.")

    def retirar(self, cantidad):
        if self._activa:
            if self._num_retiros >= 4:
                if cantidad + 1000 <= self._saldo:
                    self._saldo -= 1000
                    print("Se ha cobrado una comisión de 1000 por exceder el límite de retiros.")
                else:
                    print("Saldo insuficiente para cubrir el retiro y la comisión.")
                    return
            super().retirar(cantidad)
            self._activa = self._saldo >= 10000
        else:
            print("Cuenta inactiva. No se puede retirar.")

    def extracto_mensual(self):
        if self._num_retiros > 4:
            self._comision_mensual += (self._num_retiros - 4) * 1000
        super().extracto_mensual()
        self._activa = self._saldo >= 10000

    def imprimir(self):
        super().imprimir()
        print(f"Número total de transacciones: {self._num_consignaciones + self._num_retiros}")


In [3]:
class CuentaCorriente(Cuenta):
    def __init__(self, saldo, tasa_anual):
        super().__init__(saldo, tasa_anual)
        self._sobregiro = 0.0

    def retirar(self, cantidad):
        if cantidad > self._saldo:
            self._sobregiro += (cantidad - self._saldo)
            self._saldo = 0
            print("Retiro realizado con sobregiro.")
        else:
            super().retirar(cantidad)

    def consignar(self, cantidad):
        if self._sobregiro > 0:
            if cantidad > self._sobregiro:
                cantidad -= self._sobregiro
                self._sobregiro = 0
                super().consignar(cantidad)
            else:
                self._sobregiro -= cantidad
        else:
            super().consignar(cantidad)

    def extracto_mensual(self):
        super().extracto_mensual()

    def imprimir(self):
        super().imprimir()
        print(f"Número total de transacciones: {self._num_consignaciones + self._num_retiros}")
        print(f"Sobregiro: {self._sobregiro:.2f}")


In [4]:
def solicitar_dato(tipo, mensaje):
    while True:
        dato = input(mensaje)
        try:
            return tipo(dato)
        except ValueError:
            print(f"Entrada inválida. Por favor ingrese un valor de tipo {tipo.__name__}.")

In [6]:
def menu():
    cuenta = None

    while True:
        print("\n=== MENÚ PRINCIPAL ===")
        print("1. Crear cuenta")
        print("2. Consignar dinero")
        print("3. Retirar dinero")
        print("4. Generar extracto mensual")
        print("5. Consultar información")
        print("6. Salir")

        opcion = input("Seleccione una opción: ")

        if opcion == "1":
            tipo = input("Ingrese el tipo de cuenta (ahorros/corriente): ").strip().lower()
            saldo_inicial = solicitar_dato(float, "Ingrese el saldo inicial: ")
            tasa_anual = solicitar_dato(float, "Ingrese la tasa anual (%): ")

            if tipo == "ahorros":
                cuenta = CuentaAhorros(saldo_inicial, tasa_anual)
                print("Cuenta de ahorros creada exitosamente.")
            elif tipo == "corriente":
                cuenta = CuentaCorriente(saldo_inicial, tasa_anual)
                print("Cuenta corriente creada exitosamente.")
            else:
                print("Tipo de cuenta no válido.")

        elif opcion == "2":
            if cuenta:
                cantidad = solicitar_dato(float, "Ingrese la cantidad a consignar: ")
                cuenta.consignar(cantidad)
            else:
                print("Primero debe crear una cuenta.")

        elif opcion == "3":
            if cuenta:
                cantidad = solicitar_dato(float, "Ingrese la cantidad a retirar: ")
                cuenta.retirar(cantidad)
            else:
                print("Primero debe crear una cuenta.")

        elif opcion == "4":
            if cuenta:
                cuenta.extracto_mensual()
                print("Extracto mensual generado.")
            else:
                print("Primero debe crear una cuenta.")

        elif opcion == "5":
            if cuenta:
                cuenta.imprimir()
            else:
                print("Primero debe crear una cuenta.")

        elif opcion == "6":
            print("Saliendo del programa. ¡Hasta luego!")
            break

        else:
            print("Opción no válida. Intente de nuevo.")

In [8]:
if __name__ == "__main__":
    menu()


=== MENÚ PRINCIPAL ===
1. Crear cuenta
2. Consignar dinero
3. Retirar dinero
4. Generar extracto mensual
5. Consultar información
6. Salir
Saliendo del programa. ¡Hasta luego!
